In [3]:
# Example graph represented as an adjacency dictionary
graph = {
    'A': {'B': 1, 'C': 3},
    'B': {'A': 1, 'D': 5},
    'C': {'A': 3, 'D': 1},
    'D': {'B': 5, 'C': 1, 'E': 2},
    'E': {'D': 2, 'F': 4},
    'F': {'E': 4, 'G': 3},
    'G': {'F': 3, 'H': 2},
    'H': {'G': 2, 'I': 4},
    'I': {'H': 4, 'J': 1},
    'J': {'I': 1},
}

start = 'A'
goal = 'J'

path = a_star(graph, start, goal)
if path:
    print(f'Shortest path from {start} to {goal}: {path}')
else:
    print(f'No path found from {start} to {goal}')


Shortest path from A to J: ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


In [2]:
# Time Complexity는 H에 따라 다르다.
# O(b^d), where d = depth, b = 각 노드의 하위 요소 수
# heapque를 이용하면 길을 출력할 때 reverse를 안해도 됨

class Node:
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

def heuristic(node, goal, D=1, D2=2 ** 0.5):  # Diagonal Distance
    dx = abs(node.position[0] - goal.position[0])
    dy = abs(node.position[1] - goal.position[1])
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)


def aStar(maze, start, end):
    # startNode와 endNode 초기화
    startNode = Node(None, start)
    endNode = Node(None, end)

    # openList, closedList 초기화
    openList = []
    closedList = []

    # openList에 시작 노드 추가
    openList.append(startNode)

    # endNode를 찾을 때까지 실행
    while openList:

        # 현재 노드 지정
        currentNode = openList[0]
        currentIdx = 0

        # 이미 같은 노드가 openList에 있고, f 값이 더 크면
        # currentNode를 openList안에 있는 값으로 교체
        for index, item in enumerate(openList):
            if item.f < currentNode.f:
                currentNode = item
                currentIdx = index

        # openList에서 제거하고 closedList에 추가
        openList.pop(currentIdx)
        closedList.append(currentNode)

        # 현재 노드가 목적지면 current.position 추가하고
        # current의 부모로 이동
        if currentNode == endNode:
            path = []
            current = currentNode
            while current is not None:
                # maze 길을 표시하려면 주석 해제
                # x, y = current.position
                # maze[x][y] = 7 
                path.append(current.position)
                current = current.parent
            return path[::-1]  # reverse

        children = []
        # 인접한 xy좌표 전부
        for newPosition in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:

            # 노드 위치 업데이트
            nodePosition = (
                currentNode.position[0] + newPosition[0],  # X
                currentNode.position[1] + newPosition[1])  # Y
                
            # 미로 maze index 범위 안에 있어야함
            within_range_criteria = [
                nodePosition[0] > (len(maze) - 1),
                nodePosition[0] < 0,
                nodePosition[1] > (len(maze[len(maze) - 1]) - 1),
                nodePosition[1] < 0,
            ]

            if any(within_range_criteria):  # 하나라도 true면 범위 밖임
                continue

            # 장애물이 있으면 다른 위치 불러오기
            if maze[nodePosition[0]][nodePosition[1]] != 0:
                continue

            new_node = Node(currentNode, nodePosition)
            children.append(new_node)

        # 자식들 모두 loop
        for child in children:

            # 자식이 closedList에 있으면 continue
            if child in closedList:
                continue

            # f, g, h값 업데이트
            child.g = currentNode.g + 1
            child.h = ((child.position[0] - endNode.position[0]) **
                       2) + ((child.position[1] - endNode.position[1]) ** 2)
            # child.h = heuristic(child, endNode) 다른 휴리스틱
            # print("position:", child.position) 거리 추정 값 보기
            # print("from child to goal:", child.h)
            
            child.f = child.g + child.h

            # 자식이 openList에 있으고, g값이 더 크면 continue
            if len([openNode for openNode in openList
                    if child == openNode and child.g > openNode.g]) > 0:
                continue
                    
            openList.append(child)


def main():
    # 1은 장애물
    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

    start = (0, 0)
    end = (7, 6)

    path = aStar(maze, start, end)
    print(path)


if __name__ == '__main__':
    main()
    # [(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5), (7, 6)]
    # 실행 시간 : 0.0013초


[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5), (7, 6)]


Path found: [(0, 0), (0, 1), (0, 2), (1, 2)]


In [68]:
from math import sqrt

# List of all edges
all_edges = list(range(20))

# List of all end (unloading) nodes
end_nodes = [0,7,16,19]

# List of all directly connected paths
# All direct paths have a distance of 1
all_paths = [
    (19, 18),
    (19, 14),
    (18, 13),
    (18, 17),
    (17, 12),
    (17, 16),
    (16, 11),
    (16, 15),
    (16, 17),
    (15, 10),
    (10, 11),
    (10, 5),
    (11, 12),
    (13, 14),
    (14, 9),
    (13, 8),
    (12, 7),
    (11, 6),
    (5, 6),
    (6, 7),
    (7, 8),
    (8, 9),
    (4, 9),
    (3, 8),
    (3, 4),
    (7, 2),
    (2, 3),
    (6, 1),
    (1, 2),
    (0, 1),
    (0, 5)
]
# Add return versions of the paths
for (a,b) in all_paths.copy():
    all_paths.append((b,a))

# Costs of all paths
# All direct paths have a distance of 5
distance_cost = 1
all_path_costs = {
    x: distance_cost for x in all_paths
}

# Dictionary of nodes and their relative coordinates
all_coords = {
    0: (0, 0),
    1: (1, 0),
    2: (2, 0),
    3: (3, 0),
    4: (4, 0),
    5: (0, 1),
    6: (1, 1),
    7: (2, 1),
    8: (3, 1),
    9: (4, 1),
    10: (0, 2),
    11: (1, 2),
    12: (2, 2),
    13: (3, 2),
    14: (4, 2),
    15: (0, 3),
    16: (1, 3),
    17: (2, 3),
    18: (3, 3),
    19: (4, 3)
}

# Dijkstar Algorithm, used in A* as g()
def dijkstar(start_node, end_node):
    # Ensure proper formatting
    start_node = int(start_node)
    end_node = int(end_node)

    # Skip if the source and destination are the same
    if start_node == end_node:
        return 0

    # Initial vertex distances are set to float("inf")inity
    dijk_vertices = {
        x: float("inf") for x in all_edges if x != start_node
    }
    # Updating known direct path costs
    for (a,b) in all_paths:
        if a == start_node:
            dijk_vertices[b] = all_path_costs[(a,b)]

    # List that contains unchecked nodes
    not_checked = list(dijk_vertices.keys())
    # The loop will continue until all nodes have been checked
    while not_checked:
        # Selection of unchecked node with the lowest cost
        min_node = -1
        min_val = float("inf")
        for node in not_checked.copy():
            node_val = dijk_vertices[node]
            if node_val < min_val:
                min_val = node_val
                min_node = node
        # Updating vertex distances whenever a lower value is found
        for (a,b) in all_paths:
            if a == min_node and b != start_node:
                target_val = dijk_vertices[b]
                new_val = dijk_vertices[a] + all_path_costs[(a,b)]
                if new_val < target_val:
                    dijk_vertices[b] = new_val
        # Take off the unchecked list
        not_checked.remove(min_node)

    return dijk_vertices[end_node]

# Heuristic Algorithm, used in A* as h()
# A simple Euclidean distance calculator
def heuristic(start_node, end_node):
    # Ensure proper formatting
    start_node = int(start_node)
    end_node = int(end_node)

    # Skip if the source and destination are the same
    if start_node == end_node:
        return 0

    # Obtain the coordinates for the given nodes
    start_coord = all_coords[start_node]
    end_coord = all_coords[end_node]
    # Obtain Euclidian distances
    result = (start_coord[0] - end_coord[0]) ** 2 + \
             (start_coord[1] - end_coord[1]) ** 2
    result = sqrt(result)
    return result

# A* Algorithm
def a_star(start_node, end_node):
    # Ensure proper formatting
    start_node = int(start_node)
    end_node = int(end_node)

    # Signal to end search when target is reached
    target_reached = False
    # The A* path
    current_path = []
    # List of all checked nodes
    passed = [start_node]
    # Intermediate node checked through the loop
    latest_node = start_node
    while not target_reached:
        # Placeholder variable to find the minimum value
        min_score = float("inf")
        best_node = -1
        for (a,b) in all_paths:
            # Finding relevant paths that do not include checked nodes
            if a == latest_node and b not in passed:
                middle_node = b
                g = dijkstar(latest_node, middle_node)
                h = heuristic(middle_node, end_node)
                # The A* score
                f = g + h
                # Recording the node with the best A* score
                if f < min_score:
                    min_score = f
                    best_node = b
        # If stuck on a dead-end, return to previous node
        if best_node == -1:
            latest_node = passed[-2]
            current_path.pop(-1)
            continue
        # Add the new intermediate node to the path
        current_path.append((latest_node, best_node))
        passed.append(best_node)
        latest_node = best_node
        # If the target has been found, end the search
        if best_node == end_node:
            target_reached = True

    return current_path

def comp(initial, final, transfer):
    paths=[]
    for i in range(len(transfer)):
        paths.append(a_star(initial, transfer[i]))

In [71]:
# apply A star algorithm
import itertools


permuts = list(itertools.permutations([16,7],2))
# print(permuts)

initial=0
final=19

best_path = []
temp_path = []
best_permut = tuple()

best_cost = float("inf")
for _permut in permuts:
    total_cost = 0
    for i, n in enumerate(_permut):
        if i == 0:
            _path = a_star(initial, n)
            total_cost += len(_path)
            temp_path.append(_path)
        else:
            _path = a_star(_permut[i-1], n)
            total_cost += len(_path)
            temp_path.append(_path)
            
    _path = a_star(n,final)
    total_cost += len(_path)
    temp_path.append(_path)
        
    if total_cost < best_cost:
        best_cost = total_cost
        best_path = temp_path
        best_permut = _permut
  
    temp_path = []
        

print(f'best path: {initial} -> {best_permut} -> {final}')
print(f'best path_detial: {best_path}')
print(f'best cost: {best_cost}')

best path: 0 -> (7, 16) -> 19
best path_detial: [[(0, 1), (1, 2), (2, 7)], [(7, 12), (12, 17), (17, 16)], [(16, 17), (17, 18), (18, 19)]]
best cost: 9
